This nb is written in python3. The nb has to be run WITHOUT the source of the BigDFT package, otherwise the import yaml gives rise to a conflict.

# Comparison of results

In [1]:
import yaml
import numpy as np
AuToA = 0.5291772085**3

## Load the BG and the refernce data

In [2]:
# Functions to rearrange the BG data using the psp as the leading parameter
def get_field(full_results,study):
    """
    Retrieve the value of the field that provided the tolerance of the options
    """
    tol=full_results['options']['atol']
    field=full_results[study]['alpha_convergence']
    if field is not None: field=field[0]['f']
    return tol,field
def alpha_is_reliable(full_results,study):
    """
    Provide a boolean stating if the given study has a reliable statical polarizability
    """
    tol,field=get_field(full_results,study)
    if field is None or field == 0.0001: return False
    return True

def study_matches(study,key):
    """
    Returns the rest of the tuple of the study
    """
    #case of a string
    if type(key) == type('a'): return [i for i in study if i != key]
    #case of a tuple
    return [i for i in study if i not in list(key)]

def get_result(full_results,study,key):
    null=(None,None,None)
    if study == 'options' or not alpha_is_reliable(full_results,study): return null
    dataname=study_matches(study,key)
    th=2 if type(key) == type('a') else 3-len(key)
    if len(dataname)> th: return null
    result=full_results[study]['alpha_convergence']
    tol,field=get_field(full_results,study)
    return result,dataname,field

def reformat_result(full_results,key):
    """
    Works with a name of a (psp) as key
    """
    res={}
    import numpy
    for study in full_results:
        result,dataname,field = get_result(full_results,study,key)
        if result is None: continue
        alpha=numpy.array(result[1])#*AuToA
        st=tuple(dataname)
        mol=st[0]
        xc=st[1] if st[1] != 'lda_pw' else 'lda-SPW92'
        res.setdefault(mol,{})
        res[mol].update({xc:alpha,xc+'-f':field})
    return res

In [89]:
HG_data=yaml.load(open('HG Dataset/hg_data.yaml'))
HG_data['H']['spin_pol'] = 'sp'

MWdata=yaml.load(open('mw_data_0001v2.yaml'))
MW_data={mol:{xcref: MWdata[mol][xc]['diagonal'] \
              for xcref,xc in [('lda-SPW92','lda'),('pbe','pbe')]} for mol in MWdata}

full_results = yaml.load(open('Calculations/full_results.yaml'))

BG_data={}
for psp in ['hgh_k','nlcc_aw','nlcc_ss']:
    BG_data[psp]=reformat_result(full_results,(psp))

Define the list of the nsp and sp molecules used the attribution made in HG dataset

In [90]:
molecule_nsp = []
molecule_sp = []
for mol in HG_data:
    if HG_data[mol]['spin_pol'] == 'nsp' : molecule_nsp.append(mol)
    if HG_data[mol]['spin_pol'] == 'sp' : molecule_sp.append(mol)
molecule_nsp = sorted(molecule_nsp)
molecule_sp = sorted(molecule_sp)

In [91]:
# Routines to compute the errors
def relative_error(alpha,alpha_ref):
    """
    Provide the relative error
    """
    alpha_ref = np.array(alpha_ref)/AuToA
    return 100.0*(alpha-alpha_ref)/alpha_ref


## PSP = hgh_k

In [92]:
data = BG_data['hgh_k']
molecule_bg = list(data.keys())
ref = MW_data
molecule_ref = list(ref.keys())

In [93]:
data['NO']

{'lda-SPW92': array([ 9.832377 ,  9.832377 , 16.0643035]),
 'lda-SPW92-f': 0.01,
 'pbe': array([ 9.7822495,  9.7822495, 16.222198 ]),
 'pbe-f': 0.01,
 'pbe0': array([ 9.3536805,  9.3536805, 15.465535 ]),
 'pbe0-f': 0.01}

In [94]:
np.array(ref['no']['pbe'])/AuToA

array([ 9.77822692, 11.34321721, 16.17884364])

Some molecules are completely absent (but in some other cases converged is reached only for some of the xc)

In [82]:
for mol in molecule_nsp+molecule_sp:
    if mol not in molecule_bg:
        print(mol)

BN
CH3O
FH-OH
H2O-Li
Li2
OH


If the highest component of the relative error is greater (in modulus) than the 1% print the associated study

In [84]:
tol = 5.
xcs = ['lda-SPW92','pbe']
for xc in xcs:
    for mol in molecule_bg:
        if mol.lower() in molecule_ref:
            if xc in data[mol]:
                a = data[mol][xc]
                a_ref = ref[mol.lower()][xc]
                err = relative_error(a,a_ref)
                if np.max(np.abs(err)) > tol:
                    print(mol,xc,err)
            else:
                print(mol,xc,' not computed')
            

HO2 lda-SPW92 [ 8.63374911  4.68361008 -9.76457341]
NO lda-SPW92 [  0.20277828 -12.71023128   0.28450784]
NaCN lda-SPW92 [-3.47496044 -5.40371794 -1.98253719]
NaCl lda-SPW92 [-0.34201838 -0.34201839 -5.37147614]
OCl lda-SPW92 [ 0.40699611 -6.74974754  0.60581672]
OF lda-SPW92 [ 0.06281782 -6.92652484  0.57052459]
SCl lda-SPW92 [ 0.18829126 -8.46693999  0.37214949]
SF lda-SPW92 [ -0.19722138 -13.90047307   0.23595501]
BeH pbe  not computed
C2H pbe  not computed
HO2 pbe [  8.64864464   6.45554578 -10.19711305]
NO pbe [  0.0411381  -13.7612432    0.26796948]
OCl pbe [ 0.47570435 -6.65071455  0.56499067]
OF pbe [-0.14140664 -7.30135885  0.37409832]
SCl pbe [ 0.12618988 -8.60438528  0.27279803]
SF pbe [ -0.25143252 -14.16836186   0.20470377]


For some molecules the error is high only in one direction. Many of the y components are very high!

There is something really strange, for instance the posinp of NO is along the z axis why the y error is high and the x one is low?
